In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Define the directories for training and testing data
train_dir = 'path/to/train/directory'
test_dir = 'path/to/test/directory'

In [ ]:
# Define the image size and batch size
img_size = (256, 256)
batch_size = 32

In [ ]:
# Define the data augmentation for training data
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [ ]:
# Load the training data from the directory
train_data = train_data_gen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

In [ ]:
# Define the data augmentation for validation data
val_data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
# Load the validation data from the directory
val_data = val_data_gen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Define the CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples//batch_size,
    validation_data=val_data,
    validation_steps=val_data.samples//batch_size,
    epochs=20
)

In [ ]:
# Evaluate the model
test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

model.evaluate(test_data)

In [ ]:
# Save the model
model.save('potato_disease_classifier.h5')

In [ ]:

# Load the trained model
model = load_model('potato_disease_classifier.h5')

In [ ]:
# Load the new image of a potato
img_path = 'path/to/new/image.jpg'
img = image.load_img(img_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

In [ ]:
# Predict the disease of the new image
prediction = model.predict(img_array)

if prediction[0][0] > prediction[0][1]:
    print('The potato is healthy.')
else:
    print('The potato has a disease.')